# Evaluating NADA

Code for evaluating NADA.

Prepare the data and evaluator, load the predictions depending on the prompt construction (template or caption), and perform evaluation.

## Imports

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.evaluation import COCOEvaluator
from detectron2.structures import Instances, Boxes
import torch
from tqdm.auto import tqdm

import os
import json
from pprint import pprint

from data import setup_artdl, setup_iconart
from coco_eval import CustomIOUCOCOEvaluator

## Preparing data

In [ ]:
dataset_name = ''
# you'll  most likely in interseted in 'artdl_test' and 'iconart_test'
# see data.py for more options

setup_artdl()
setup_iconart()

dataset = DatasetCatalog.get(dataset_name)
metadata = MetadataCatalog.get(dataset_name)

## Prepare evaluator

In [ ]:
evaluator = COCOEvaluator(dataset_name, output_dir='results')

# uncomment the line below if you want a custom IOU not shown by default by COCOEvaluator or if you want per-class results that aren't AP 0.50:0.95
# evaluator = CustomIOUCOCOEvaluator(dataset_name, iou=0.5, output_dir=output_dir)

## Loading predictions

### Template prompt construction

In [ ]:
inputs = []
outputs = []
missing_preds = []
incomplete_annotations = []

all_class_preds_filepath = ''
all_det_preds_filepath = ''

for item in tqdm(dataset):
  pred_boxes, scores, pred_classes = [], [], []

  with open(os.path.join(
      all_class_preds_filepath,
      f"{item['image_id']}.json"
  )) as f:
    labels = json.load(f)['labels']

  for label in labels:
    label_id = metadata.thing_classes.index(label)

    pred_filepath = os.path.join(
        all_det_preds_filepath,
        label,
        item['image_id']+'.json'
    )

    if os.path.exists(pred_filepath):
      annotations_exist = True

      with open(pred_filepath) as f:
        preds = json.load(f)
        for pred_ann in preds['annotations']:
          x, y, w, h = pred_ann['bbox']
          pred_boxes.append([x, y, x+w, y+h])
          scores.append(1.0)
          pred_classes.append(pred_ann['id'])
    else:
      missing_preds.append(pred_filepath)

  annotations_exist = (len(pred_boxes) == len(scores) == len(pred_classes)) and len(pred_boxes) > 0
  inputs.append(item)
  outputs.append({
      'instances': Instances(
          (item['height'], item['width']),
          pred_boxes=Boxes(torch.tensor(pred_boxes)),
          scores=torch.tensor(scores),
          pred_classes=torch.tensor(pred_classes)
      )
  })
  if not annotations_exist:
    print(item['image_id'], labels)
    incomplete_annotations.append(item['image_id'])

### Caption prompt construction

In [ ]:
from detectron2.structures import Instances, Boxes
import torch

import os
import json
from pprint import pprint
from tqdm.auto import tqdm

inputs = []
outputs = []
missing_preds = []
incomplete_annotations = []

all_caps_filepath = ''
all_det_preds_filepath = ''

for item in tqdm(dataset):
  pred_boxes, scores, pred_classes = [], [], []

  with open(os.path.join(
      all_caps_filepath,
      f"{item['image_id']}.json"
  )) as f:
    labels = [list(label_caption.keys())[0] for label_caption in json.load(f)['prompts_for_generation']]

  for label in labels:
    label_id = metadata.thing_classes.index(label)

    pred_filepath = os.path.join(
        all_det_preds_filepath,
        label,
        item['image_id']+'.json'
    )

    if os.path.exists(pred_filepath):
      annotations_exist = True

      with open(pred_filepath) as f:
        preds = json.load(f)
        for pred_ann in preds['annotations']:
          x, y, w, h = pred_ann['bbox']
          pred_boxes.append([x, y, x+w, y+h])
          scores.append(1.0)
          pred_classes.append(pred_ann['id'])
    else:
      missing_preds.append(pred_filepath)

  annotations_exist = (len(pred_boxes) == len(scores) == len(pred_classes)) and len(pred_boxes) > 0
  inputs.append(item)
  outputs.append({
      'instances': Instances(
          (item['height'], item['width']),
          pred_boxes=Boxes(torch.tensor(pred_boxes)),
          scores=torch.tensor(scores),
          pred_classes=torch.tensor(pred_classes)
      )
  })
  if not annotations_exist:
    print(item['image_id'], labels)
    incomplete_annotations.append(item['image_id'])

## Evaluating predictions

In [ ]:
evaluator.reset()
evaluator.process(inputs, outputs)
results = evaluator.evaluate()